In [ ]:
import h5py
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os

from uoineuro.coupling_utils import *

%matplotlib inline

In [ ]:
results_path = os.path.join(os.environ['HOME'],
                            'fits/uoineuro/coupling/nhp/nhp_indy_20160411_01_coupling.h5')

In [ ]:
results = h5py.File(results_path, 'r')

# Compare UoI Poisson / UoI Lasso

In [ ]:
uoi_poisson_coefs = np.median(results['uoi_poisson_bic/coupling_coefs'][:], axis=0)
uoi_lasso_coefs = np.median(results['uoi_lasso_bic/coupling_coefs'][:], axis=0)
glmnet_coefs = np.median(results['glmnet_poisson/coupling_coefs'][:], axis=0)
lasso_coefs = np.median(results['lasso/coupling_coefs'][:], axis=0)

In [ ]:
np.unique(uoi_poisson_coefs[8])

In [ ]:
correlations_uoi = coupling_coef_corrs(
    uoi_poisson_coefs,
    uoi_lasso_coefs
)

In [ ]:
correlations_baseline = coupling_coef_corrs(
    glmnet_coefs,
    lasso_coefs
)

In [ ]:
plt.hist(correlations_baseline, alpha=0.5, color='gray')
plt.hist(correlations_uoi, alpha=0.5, color='red')

In [ ]:
plt.scatter(correlations_baseline, correlations_uoi)
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.plot([0, 1], [0, 1], color='k')

In [ ]:
probs_uoi = selection_profiles_by_chance(
    uoi_poisson_coefs,
    uoi_lasso_coefs 
)

In [ ]:
probs_baseline = selection_profiles_by_chance(
    glmnet_coefs,
    lasso_coefs 
)

In [ ]:
probs_uoi[probs_uoi == 0] = 1e-16
probs_baseline[probs_baseline == 0] = 1e-16

plt.hist(np.log(probs_baseline), alpha=0.5, color='gray')
plt.hist(np.log(probs_uoi), alpha=0.5, color='red')

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(8, 5))
axes[0].hist(corrs, color='gray')
axes[0].set_xlim([0, 1])
axes[0].set_title(r'\textbf{Correlation between UoI}$_{\textbf{Poisson}}$\textbf{ and UoI}$_{\textbf{Lasso}}$', fontsize=23)

probs1[probs1 == 0] = 1e-16
axes[1].hist(np.log(probs1), color='gray', bins=np.arange(-40, 0, 5))
axes[1].axvline(np.log(0.1), color='r')
axes[1].axvline(np.log(0.001), color='b')
axes[1].set_xlim([0, -40])
axes[1].set_title(r'\textbf{Log-probabilities of Matching Sel. Profile}', fontsize=23)
plt.tight_layout()
#plt.savefig('poisson_graph_comp.pdf', bbox_inches='tight')

In [ ]:
probs1 = selection_profiles_by_chance(
    fits_path=fits_path,
    dataset1='UoI_Poisson_log',
    dataset2='UoI_Lasso_r2'
)

probs2 = selection_profiles_by_chance(
    fits_path=fits_path,
    dataset1='UoI_Lasso_r2',
    dataset2='UoI_Poisson_log'
)

In [ ]:
G_uoi_poisson = create_graph(fits_path, dataset='UoI_Poisson_log', directed=True)
G_uoi_lasso = create_graph(fits_path, dataset='UoI_Lasso_r2', directed=True)
G_lasso = create_graph(fits_path, dataset='Lasso', directed=True)

In [ ]:
Gs = [G_uoi_poisson, G_uoi_lasso, G_lasso]
closeness = [np.array(list(nx.algorithms.centrality.closeness_centrality(G).values())) for G in Gs]
eigenvec = [np.array(list(nx.algorithms.centrality.eigenvector_centrality_numpy(G).values())) for G in Gs]
betweenness = [np.array(list(nx.algorithms.centrality.betweenness_centrality(G).values())) for G in Gs]
katz = [np.array(list(nx.algorithms.centrality.katz_centrality_numpy(G, alpha=0.02).values())) for G in Gs]
indeg = [np.array(list(dict(G.in_degree()).values())) for G in Gs]
outdeg = [np.array(list(dict(G.out_degree()).values())) for G in Gs]
measures = [closeness, betweenness, indeg, eigenvec, katz, outdeg]

In [ ]:
from scipy.stats import mannwhitneyu

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

axes[0, 0].scatter(closeness[0], closeness[1], color='r', alpha=0.5)
axes[0, 0].scatter(closeness[2], closeness[1], color='gray', alpha=0.5)
axes[0, 0].plot([0.5, 0.7], [0.5, 0.70], color='k')
axes[0, 0].set_xlim([0.5, 0.7])
axes[0, 0].set_ylim([0.5, 0.7])
axes[0, 0].set_xticks([0.5, 0.6, 0.7])
axes[0, 0].set_yticks([0.5, 0.6, 0.7])
axes[0, 0].set_title(r'\textbf{Closeness Centrality}', fontsize=20)


axes[1, 0].scatter(eigenvec[0], eigenvec[1], color='r', alpha=0.5)
axes[1, 0].scatter(eigenvec[2], eigenvec[1], color='gray', alpha=0.5)
axes[1, 0].plot([0, 0.2], [0, 0.2], color='k')
axes[1, 0].set_xlim([0, 0.2])
axes[1, 0].set_ylim([0, 0.2])
axes[1, 0].set_xticks([0, 0.1, 0.2])
axes[1, 0].set_yticks([0, 0.1, 0.2])
axes[1, 0].set_title(r'\textbf{Eigenvector Centrality}', fontsize=20)


axes[0, 1].scatter(betweenness[0], betweenness[1], color='r', alpha=0.5)
axes[0, 1].scatter(betweenness[2], betweenness[1], color='gray', alpha=0.5)
axes[0, 1].plot([0, 0.03], [0, 0.03], color='k')
axes[0, 1].set_xlim([0, 0.03])
axes[0, 1].set_ylim([0, 0.03])
axes[0, 1].set_xticks([0, 0.01, 0.02, 0.03])
axes[0, 1].set_yticks([0, 0.01, 0.02, 0.03])
axes[0, 1].set_xticklabels(['0.0', '.01', '.02', '.03'])
axes[0, 1].set_yticklabels(['0.0', '.01', '.02', '.03'])
axes[0, 1].set_title(r'\textbf{Betweeness Centrality}', fontsize=20)


axes[1, 1].scatter(katz[0], katz[1], color='r', alpha=0.5)
axes[1, 1].scatter(katz[2], katz[1], color='gray', alpha=0.5)
axes[1, 1].plot([0, 0.2], [0, 0.2], color='k')
axes[1, 1].set_xlim([0, 0.2])
axes[1, 1].set_ylim([0, 0.2])
axes[1, 1].set_xticks([0, 0.1, 0.2])
axes[1, 1].set_yticks([0, 0.1, 0.2])
axes[1, 1].set_title(r'\textbf{Katz Centrality}', fontsize=20)


axes[0, 2].scatter(indeg[0], indeg[1], color='r', alpha=0.5)
axes[0, 2].scatter(indeg[2], indeg[1], color='gray', alpha=0.5)
axes[0, 2].plot([0, 100], [0, 100], color='k')
axes[0, 2].set_xlim([0, 100])
axes[0, 2].set_ylim([0, 100])
axes[0, 2].set_xticks([0, 25, 50, 75, 100])
axes[0, 2].set_yticks([0, 25, 50, 75, 100])
axes[0, 2].set_title(r'\textbf{In-Degree Distribution}', fontsize=20)

axes[1, 2].scatter(outdeg[0], outdeg[1], color='r', alpha=0.5)
axes[1, 2].scatter(outdeg[2], outdeg[1], color='gray', alpha=0.5)
axes[1, 2].plot([0, 100], [0, 100], color='k')
axes[1, 2].set_xlim([0, 100])
axes[1, 2].set_ylim([0, 100])
axes[1, 2].set_xticks([0, 25, 50, 75, 100])
axes[1, 2].set_yticks([0, 25, 50, 75, 100])
axes[1, 2].set_title(r'\textbf{Out-Degree Distribution}', fontsize=20)

for idx, ax in enumerate(axes.ravel()):
    ax.set_aspect('equal')
    corr1 = np.corrcoef(measures[idx][0], measures[idx][1])[0, 1]
    corr2 = np.corrcoef(measures[idx][1], measures[idx][2])[0, 1]
    fig.text(x=0.68, y=0.15, s=r'$r=%0.2f$' %corr1, color='red', transform=ax.transAxes, fontsize=20)
    fig.text(x=0.68, y=0.05, s=r'$r=%0.2f$' %corr2, color='gray', transform=ax.transAxes, fontsize=20)
    
plt.tight_layout()
fig.text(
    x=0.52, y=-0.02, s=r'\textbf{UoI}$_{\text{Poisson}}$\textbf{ (red) / Lasso (Gray)}', fontsize=30,
    horizontalalignment='center', verticalalignment='center')

fig.text(
    x=-0.02, y=0.5, s=r'\textbf{UoI}$_{\text{Lasso}}$', fontsize=30,
    horizontalalignment='center', verticalalignment='center', rotation=90
)

plt.savefig('graph_metrics.pdf', bbox_inches='tight')